In [1]:
import script
from flask import Flask, jsonify, request
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import json


In [2]:
app = Flask(__name__)

In [3]:
@app.route('/api/machine_learning', methods = ['GET'])
def get_machine_learning():
    args = request.args
    id = args.get("id")
    recipe_list = script.main_function(int(id))
    return {'recipe_list': recipe_list}, 200


In [4]:
@app.route('/api/tags', methods = ['GET'])
def get_tags():
    smallRecipes = pd.read_csv('../src/recipesDB/Small_Recipes.csv')
    def returnSteps(smallRecipe):
        smallRecipe = pd.DataFrame(smallRecipe.set_index('id')['tags'])
        smallRecipe["tags"] = smallRecipe["tags"].str.replace("[", '')
        smallRecipe["tags"] = smallRecipe["tags"].str.replace("']", '')
        smallRecipe = smallRecipe.tags.str.split("', '",expand=True)
        smallRecipe[0] = smallRecipe[0].str.replace("'", '')
        return smallRecipe
    allSteps = returnSteps(smallRecipes)
    tags = []
    for i in range(len(allSteps.columns)):
        tags = tags + list(allSteps.iloc[:,i].unique())
    tags = set(tags)
    while '' in tags:
        tags.remove('')
    while None in tags:
        tags.remove(None)
    tags = list(tags)

    return {'tags': tags}, 200
    

In [5]:
smallRecipes = pd.read_csv('../src/recipesDB/Small_Recipes.csv')
def returnSteps(smallRecipe):
    smallRecipe = pd.DataFrame(smallRecipe.set_index('id')['tags'])
    smallRecipe["tags"] = smallRecipe["tags"].str.replace("[", '')
    smallRecipe["tags"] = smallRecipe["tags"].str.replace("']", '')
    smallRecipe = smallRecipe.tags.str.split("', '",expand=True)
    smallRecipe[0] = smallRecipe[0].str.replace("'", '')
    return smallRecipe
allSteps = returnSteps(smallRecipes)
tags = []
for i in range(len(allSteps.columns)):
    tags = tags + list(allSteps.iloc[:,i].unique())
tags = set(tags)
while '' in tags:
    tags.remove('')
while None in tags:
    tags.remove(None)
s = list(tags)
s

C:\Users\sushi\AppData\Local\Temp\ipykernel_4556\2759432294.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  smallRecipe["tags"] = smallRecipe["tags"].str.replace("[", '')
C:\Users\sushi\AppData\Local\Temp\ipykernel_4556\2759432294.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  smallRecipe["tags"] = smallRecipe["tags"].str.replace("']", '')


['served-hot',
 'picnic',
 'lettuces',
 'pies-and-tarts',
 'spring',
 'swiss',
 'appetizers',
 'clams',
 'dietary',
 'french',
 'scottish',
 'cheese',
 'pancakes-and-waffles',
 'rosh-hashanah',
 'tuna',
 'cookies-and-brownies',
 'british-columbian',
 'course',
 'filipino',
 'elk',
 'salmon',
 'spreads',
 'irish',
 'fall',
 'camping',
 'no-cook',
 'steak',
 'beverages',
 'yams-sweet-potatoes',
 'sweet-sauces',
 'coffee-cakes',
 'iranian-persian',
 'onions',
 'water-bath',
 'oamc-freezer-make-ahead',
 '1-day-or-more',
 'brown-bag',
 'occasion',
 'diabetic',
 'northeastern-united-states',
 'black-beans',
 'catfish',
 'tomatoes',
 'turkish',
 'eggs-dairy',
 'sandwiches',
 'deep-fry',
 'candy',
 'colombian',
 'manicotti',
 'midwestern',
 'main-ingredient',
 'strawberries',
 'steam',
 'zucchini',
 'smoothies',
 'lamb-sheep',
 'russian',
 'long-grain-rice',
 'brown-rice',
 'to-go',
 'superbowl',
 'moose',
 'beginner-cook',
 'south-west-pacific',
 'cherries',
 'savory-sauces',
 'nut-free',
 'c

In [6]:
"""@app.route('/api/recherche_nom', methods = ['GET'])
def get_recherche_nom():
    args = request.args
    name = args.get("name")
    df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
    df1 = df[["name","id"]].copy()
    df1 = df1.set_index("id")
    df1 = df1["name"].str.contains(name) 
    result = df1.loc[df1 == True].to_frame().index.to_list()
    df2 = []
    for i in result:
        recipe = script.SmallRecipe(int(i))
        df2.append(recipe.__dict__())
    return {'recipes': df2}, 200"""

'@app.route(\'/api/recherche_nom\', methods = [\'GET\'])\ndef get_recherche_nom():\n    args = request.args\n    name = args.get("name")\n    df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")\n    df1 = df[["name","id"]].copy()\n    df1 = df1.set_index("id")\n    df1 = df1["name"].str.contains(name) \n    result = df1.loc[df1 == True].to_frame().index.to_list()\n    df2 = []\n    for i in result:\n        recipe = script.SmallRecipe(int(i))\n        df2.append(recipe.__dict__())\n    return {\'recipes\': df2}, 200'

In [7]:
@app.route('/api/recette', methods = ['GET'])
def get_recette_all():
    args = request.args
    id = args.get("id")
    if id is None:
        df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
        id = list(df['id'])
        df1 = []
        for i in range(693):
            """recipe = script.SmallRecipe(int(i))
            df1.append(recipe.__dict__())"""
            print(id[i], i)
            response = requests.get('http://www.food.com/recipe/' + str(i))
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                suggestion = script.MediumRecipe(int(id[i]), soup)
                df1.append(suggestion.__dict__())
        return {'recipes': df1}, 200
    recipe = script.SmallRecipe(int(id))
    return {'recipes': recipe.__dict__()}, 200

In [8]:
@app.route('/api/utilisateur', methods = ['POST'])
def post_nouvel_utilisateur():
    df = pd.read_csv("../src/recipesDB/User.csv")
    args = request.args
    firstName = args.get("FirstName")
    lastName = args.get("LastName")
    password = args.get("Password")
    mail = args.get("Email")
    avatar = args.get("MyAvatar")
    date = datetime.today().strftime('%Y-%m-%d')
    id = df.last().id + 1
    likes = []
    user = pd.DataFrame({"id":[id],"firstName": [firstName], "lastName": [lastName], "password": [password], "mail": [mail], "avatar": [avatar], "date": [date]})
    user.to_json("../src/recipesDB/User.csv", mode='a', header=False)

In [9]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
C:\Users\sushi\AppData\Local\Temp\ipykernel_4556\1638595738.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  smallRecipe["tags"] = smallRecipe["tags"].str.replace("[", '')
C:\Users\sushi\AppData\Local\Temp\ipykernel_4556\1638595738.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  smallRecipe["tags"] = smallRecipe["tags"].str.replace("']", '')
127.0.0.1 - - [05/Jul/2022 22:30:05] "GET /api/tags HTTP/1.1" 200 -
